In [ ]:
!pip install torch torchvision
!pip install opencv-python

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import torch
import torch.nn as nn

class FSRCNN_model(nn.Module):
    def __init__(self, scale: int) -> None:
        super(FSRCNN_model, self).__init__()

        if scale not in [2, 3, 4]:
            raise ValueError("must be 2, 3 or 4")

        d = 56
        s = 12

        self.feature_extract = nn.Conv2d(in_channels=3, out_channels=d, kernel_size=5, padding=2)
        nn.init.kaiming_normal_(self.feature_extract.weight)
        nn.init.zeros_(self.feature_extract.bias)

        self.activation_1 = nn.PReLU(num_parameters=d)

        self.shrink = nn.Conv2d(in_channels=d, out_channels=s, kernel_size=1)
        nn.init.kaiming_normal_(self.shrink.weight)
        nn.init.zeros_(self.shrink.bias)

        self.activation_2 = nn.PReLU(num_parameters=s)


        self.map_1 = nn.Conv2d(in_channels=s, out_channels=s, kernel_size=3, padding=1)
        nn.init.kaiming_normal_(self.map_1.weight)
        nn.init.zeros_(self.map_1.bias)

        self.map_2 = nn.Conv2d(in_channels=s, out_channels=s, kernel_size=3, padding=1)
        nn.init.kaiming_normal_(self.map_2.weight)
        nn.init.zeros_(self.map_2.bias)

        self.map_3 = nn.Conv2d(in_channels=s, out_channels=s, kernel_size=3, padding=1)
        nn.init.kaiming_normal_(self.map_3.weight)
        nn.init.zeros_(self.map_3.bias)

        self.map_4 = nn.Conv2d(in_channels=s, out_channels=s, kernel_size=3, padding=1)
        nn.init.kaiming_normal_(self.map_4.weight)
        nn.init.zeros_(self.map_4.bias)

        self.activation_3 = nn.PReLU(num_parameters=s)

        self.expand = nn.Conv2d(in_channels=s, out_channels=d, kernel_size=1)
        nn.init.kaiming_normal_(self.expand.weight)
        nn.init.zeros_(self.expand.bias)

        self.activation_4 = nn.PReLU(num_parameters=d)

        self.deconv = nn.ConvTranspose2d(in_channels=d, out_channels=3, kernel_size=9,
                                        stride=scale, padding=4, output_padding=scale-1)
        nn.init.normal_(self.deconv.weight, mean=0.0, std=0.001)
        nn.init.zeros_(self.deconv.bias)

    def forward(self, X_in):
        X = self.feature_extract(X_in)
        X = self.activation_1(X)

        X = self.shrink(X)
        X = self.activation_2(X)

        X = self.map_1(X)
        X = self.map_2(X)
        X = self.map_3(X)
        X = self.map_4(X)
        X = self.activation_3(X)

        X = self.expand(X)
        X = self.activation_4(X)

        X = self.deconv(X)
        X_out = torch.clip(X, 0.0, 1.0)

        return X_out

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = FSRCNN_model(scale=2).to(device)
model.load_state_dict(torch.load("/content/drive/MyDrive/AI/CV/SSD/FSRCNN-x2.pt", map_location=device))

<ipython-input-4-a14c8405c861>:3: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load("/content/drive/MyDrive/AI/CV/SSD/FSRCNN-x2.pt", map_locatio

<All keys matched successfully>

In [ ]:
import cv2
import torch
import numpy as np
import time

def upscale_video(input_path, output_path, model, device, scale_factor=2):


    cap = cv2.VideoCapture(input_path)
    if not cap.isOpened():
        print("Error opening video stream or file")
        return

    width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH) * scale_factor)
    height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT) * scale_factor)
    fps = cap.get(cv2.CAP_PROP_FPS)

    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    out = cv2.VideoWriter(output_path, fourcc, fps, (width, height))

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        frame = frame.transpose((2, 0, 1))
        frame = np.ascontiguousarray(frame)

        input_tensor = torch.tensor(frame, dtype=torch.float32).unsqueeze(0).to(device) / 255.0

        with torch.no_grad():
            output_tensor = model(input_tensor)

        output_frame = output_tensor.squeeze().cpu().numpy()
        output_frame = output_frame.transpose((1, 2, 0))
        output_frame = (output_frame * 255.0).clip(0, 255).astype(np.uint8)
        output_frame = cv2.cvtColor(output_frame, cv2.COLOR_RGB2BGR)

        out.write(output_frame)

    cap.release()
    out.release()




input_video_path = "/content/drive/MyDrive/AI/CV/SSD/videoplayback.mp4"
output_video_path = "/content/drive/MyDrive/AI/CV/SSD/Output56.mp4"
upscale_video(input_video_path, output_video_path, model, device, scale_factor=2)

In [ ]:
import cv2

def videoResolution(video_path):

    cap = cv2.VideoCapture(video_path)

    if not cap.isOpened():
        return "Video can't be opened"


    height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))


    cap.release()


    if height >= 2160:
        return "2160p (4K)"
    elif height >= 1440:
        return "1440p (QHD)"
    elif height >= 1080:
        return "1080p (Full HD)"
    elif height >= 720:
        return "720p (HD)"
    elif height >= 480:
        return "480p (SD)"
    else:
        return "Lower than 480p"


In [ ]:
videoQuality = videoResolution("/content/drive/MyDrive/AI/CV/SSD/videoplayback.mp4")
print(f"Video quality of this input video is: {videoQuality}")

Video quality of this input video is: 480p (SD)


In [ ]:
videoQuality = videoResolution("/content/drive/MyDrive/AI/CV/SSD/Output56.mp4")
print(f"Video quality of this output video is: {videoQuality}")

Video quality of this output video is: 720p (HD)
